# install hf transformer

In [2]:
!pip install transformers
!pip install accelerate
!pip install evaluate
#==4.26.1 #==0.16.0

In [3]:
!pip list | grep accelerate
!pip list | grep transformers
!pip list | grep evaluate

accelerate                       0.29.3
transformers                     4.40.0
evaluate                         0.4.2


# import

In [3]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset, random_split
# from transformers import AdamW
from torch.optim import AdamW

from tqdm.auto import tqdm
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"

In [5]:
device

'cuda'

# set seed

In [4]:
# Fix random seed for reproducibility
seed = 42
def same_seeds(seed):
	torch.manual_seed(seed)
	if torch.cuda.is_available():
			torch.cuda.manual_seed(seed)
			torch.cuda.manual_seed_all(seed)
	np.random.seed(seed)
	random.seed(seed)
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True
same_seeds(seed)

# load model and tokenizer


In [5]:
from transformers import (
  BertTokenizerFast,
  AutoModel,
  AutoModelForSequenceClassification
)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = AutoModelForSequenceClassification.from_pretrained(
    'ckiplab/bert-base-chinese',
    num_labels=3
).to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ckiplab/bert-base-chinese and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# load data

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
path = "/content/drive/My Drive/ML/project/graduation/專題資料"
df = pd.read_parquet(path + "/train_data_ori.parquet")

In [7]:
df[df.isna().any(axis=1)]

,text_x,replyType


In [8]:
print(df.isna().sum())
# df.dropna(inplace=True)
print(df.isnull().sum())
df.dropna(inplace=True)
print(df.isnull().sum())
print(df.isna().sum())
len(df)

text_x       0
replyType    0
dtype: int64
text_x       0
replyType    0
dtype: int64
text_x       0
replyType    0
dtype: int64
text_x       0
replyType    0
dtype: int64


43092

# dataset

In [8]:
def tokenize_function(examples):
    return tokenizer(str(examples), padding="max_length", truncation=True)
# tokenize_function(df["text_x"][1])

In [9]:
def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set))
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return train_set, valid_set

In [10]:
# torch dataloader
label_map = {
    "NOT_RUMOR":0,
    "RUMOR":1,
    "OPINIONATED":2
}
class News_Dataset(Dataset):
    def __init__(self,df):
        self.df = df
        pass
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        sample =  {}
        text_token = tokenize_function(self.df["text_x"].iloc[idx])
        label = label_map[self.df["replyType"].iloc[idx]]
        for i in text_token:
            sample[i] = torch.tensor(text_token[i])
        sample["labels"] = label
        # print(e,"\n",text_token,label,sample)
        return sample
    def get(self, idx):
        return self.df.iloc[idx]

# train_set[0]

# train


In [10]:
torch.cuda.empty_cache()

In [11]:
# hyperparameters
validation = True
logging_step = 100
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
# optimizer = AdamW(model.parameters(), lr=learning_rate)
train_batch_size = 8

valid_ratio = 0.2

train_dataset = News_Dataset(df)

train_set, valid_set = train_valid_split(train_dataset, valid_ratio, seed)
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(valid_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)

In [17]:
# display(df.iloc[4772])
# train_dataset[4772]
# tokenize_function(df.iloc[4773])

In [18]:
# t_set = News_Dataset(df)
# t_set.get(0)
# for idx, (data) in enumerate(train_dataset):
#     print(idx)

In [13]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [20]:
# for a in train_loader:
#     print(a.keys())
    # break

In [13]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
loss_fn = torch.nn.CrossEntropyLoss()
model.train()
for epoch in range(num_epochs):
    i=0
    train_loss = np.array([])
    for b in train_loader:
    # for batch in train_loader:
        i += 1
        # print(i)
        batch = {k: v.to(device) for k, v in b.items()}
        # outputs = model(**batch)
        # input_ids = batch['input_ids']
        # try:
        # token_type_ids = batch["token_type_ids"]
        # except Exception as e:
        #     print(batch.keys(),e)
            # token_type_ids = None
        # attention_mask = batch['attention_mask']
        outputs = model(input_ids = batch['input_ids'],
                        token_type_ids=batch["token_type_ids"],
                        attention_mask = batch['attention_mask'])
        pred = outputs.logits
        loss = loss_fn(pred, batch['labels'])
        # loss = outputs.loss
        loss.backward()
        optimizer.step()

        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        train_loss = np.append(train_loss,loss.item())
        if i % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {i} | loss = {train_loss.mean():.3f}")
            train_loss = 0
        # break

    #     # train_last_loss = train_batch_loss / train_batch_size

    # train_last_loss = train_batch_loss.mean
    print(f'epoch {epoch+1} train loss: {np.mean(train_loss)}')
    test_loss = np.array([])
    model.eval()
    for b in test_loader:
        batch = {k: v.to(device) for k, v in b.items()}
        with torch.no_grad():
            outputs = model(input_ids = batch['input_ids'],
                        token_type_ids=batch["token_type_ids"],
                        attention_mask = batch['attention_mask'])

        # Logits act as predictions
        pred = outputs.logits
        loss = loss_fn(pred, batch['labels'])
        test_loss = np.append(test_loss,loss.item())
        # break
    print(f'epoch {epoch+1} test loss: {np.mean(test_loss)}')
    path = f"/content/drive/My Drive/ML/project/graduation/models/bert_base_notime/base_{epoch}.pth"
    torch.save(model, path)

  0%|          | 0/12930 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 0.946
Epoch 1 | Step 200 | loss = 0.828
Epoch 1 | Step 300 | loss = 0.826
Epoch 1 | Step 400 | loss = 0.836
Epoch 1 | Step 500 | loss = 0.804
Epoch 1 | Step 600 | loss = 0.755
Epoch 1 | Step 700 | loss = 0.765
Epoch 1 | Step 800 | loss = 0.719
Epoch 1 | Step 900 | loss = 0.759
Epoch 1 | Step 1000 | loss = 0.721
Epoch 1 | Step 1100 | loss = 0.702
Epoch 1 | Step 1200 | loss = 0.768
Epoch 1 | Step 1300 | loss = 0.720
Epoch 1 | Step 1400 | loss = 0.720
Epoch 1 | Step 1500 | loss = 0.705
Epoch 1 | Step 1600 | loss = 0.726
Epoch 1 | Step 1700 | loss = 0.726
Epoch 1 | Step 1800 | loss = 0.716
Epoch 1 | Step 1900 | loss = 0.714
Epoch 1 | Step 2000 | loss = 0.698
Epoch 1 | Step 2100 | loss = 0.697
Epoch 1 | Step 2200 | loss = 0.668
Epoch 1 | Step 2300 | loss = 0.707
Epoch 1 | Step 2400 | loss = 0.696
Epoch 1 | Step 2500 | loss = 0.664
Epoch 1 | Step 2600 | loss = 0.713
Epoch 1 | Step 2700 | loss = 0.671
Epoch 1 | Step 2800 | loss = 0.675
Epoch 1 | Step 2900 | loss = 

# evaluate

## acc

In [13]:
import evaluate
for i in range(3):
    mode = torch.load(f"/content/drive/My Drive/ML/project/graduation/models/bert_base_notime/base_{i}.pth")
    model = mode.to(device)
    eval = ["accuracy"]
    for eva in eval:
        metric = evaluate.load(eva)
        model.eval()
        for batch in tqdm(test_loader):
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(input_ids = batch['input_ids'],
                            token_type_ids=batch["token_type_ids"],
                            attention_mask = batch['attention_mask'])

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            metric.add_batch(predictions=predictions, references=batch["labels"])

        print(f"test {eva}: ",metric.compute())
        metric = evaluate.load(eva)
        model.eval()
        for batch in tqdm(train_loader):
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(input_ids = batch['input_ids'],
                            token_type_ids=batch["token_type_ids"],
                            attention_mask = batch['attention_mask'])

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            metric.add_batch(predictions=predictions, references=batch["labels"])

        print(f"train {eva}: ",metric.compute())

  0%|          | 0/1078 [00:00<?, ?it/s]

test accuracy:  {'accuracy': 0.7231376189371084}


  0%|          | 0/4310 [00:00<?, ?it/s]

train accuracy:  {'accuracy': 0.7538434762429657}


  0%|          | 0/1078 [00:00<?, ?it/s]

...

train accuracy:  {'accuracy': 0.8377907988629112}


  0%|          | 0/1078 [00:00<?, ?it/s]

test accuracy:  {'accuracy': 0.7212810396843815}


  0%|          | 0/4310 [00:00<?, ?it/s]

train accuracy:  {'accuracy': 0.9288449266113593}


## f1

In [ ]:
from sklearn.metrics import f1_score

In [13]:
# f1 score
import evaluate
for method in ["macro","micro"]:
    for i in range(3):
        mode = torch.load(f"/content/drive/My Drive/ML/project/graduation/models/bert_base_notime/base_{i}.pth")
        model = mode.to(device)
        eval = ["f1"]
        for eva in eval:
            metric = evaluate.load(eva)
            model.eval()
            for batch in tqdm(test_loader):
                batch = {k: v.to(device) for k, v in batch.items()}
                with torch.no_grad():
                    outputs = model(input_ids = batch['input_ids'],
                                token_type_ids=batch["token_type_ids"],
                                attention_mask = batch['attention_mask'])

                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1)
                metric.add_batch(predictions=predictions, references=batch["labels"])
                # break
            print(f"test epoch{i} {eva} {method}: ",metric.compute(average=method))
            metric = evaluate.load(eva)
            model.eval()
            for batch in tqdm(train_loader):
                batch = {k: v.to(device) for k, v in batch.items()}
                with torch.no_grad():
                    outputs = model(input_ids = batch['input_ids'],
                                token_type_ids=batch["token_type_ids"],
                                attention_mask = batch['attention_mask'])

                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1)
                metric.add_batch(predictions=predictions, references=batch["labels"])


            print(f"test epoch{i} {eva} {method}: ",metric.compute(average=method))

  0%|          | 0/1078 [00:00<?, ?it/s]

test epoch0 f1 macro:  {'f1': 0.5780556623338006}


  0%|          | 0/4310 [00:00<?, ?it/s]

test epoch0 f1 macro:  {'f1': 0.6188647100499672}


  0%|          | 0/1078 [00:00<?, ?it/s]

...

test epoch1 f1 micro:  {'f1': 0.8377907988629112}


  0%|          | 0/1078 [00:00<?, ?it/s]

test epoch2 f1 micro:  {'f1': 0.7212810396843815}


  0%|          | 0/4310 [00:00<?, ?it/s]

test epoch2 f1 micro:  {'f1': 0.9288449266113592}


In [ ]:
# for epoch in range(num_epochs):
#     test_loss = np.array([])
#     model.eval()
#     for b in test_loader:
#         batch = {k: v.to(device) for k, v in b.items()}
#         with torch.no_grad():
#             outputs = model(input_ids = batch['input_ids'],
#                         token_type_ids=batch["token_type_ids"],
#                         attention_mask = batch['attention_mask'])

#         # Logits act as predictions
#         pred = outputs.logits
#         loss = loss_fn(pred, batch['labels'])
#         test_loss = np.append(test_loss,loss.item())
#     print(f'epoch {epoch} last loss: {np.mean(test_loss)}')

# perdiction

In [ ]:
model.eval()
batch = test_loader.dataset[0]
batch = {k: v.to(device) for k, v in batch.items()}
with torch.no_grad():
    output = model(input_ids = batch['input_ids'],
                token_type_ids=batch["token_type_ids"],
                attention_mask = batch['attention_mask'])
    logits = output.logits
    pred = torch.argmax(logits, dim=-1)
    print(pred,batch["labels"])